In [86]:
from openforcefield.topology import Molecule
import qcengine
import qcelemental as qcel
from qcelemental.models import AtomicInput
from qcelemental.models.common_models import Model
from openeye import oequacpac, oechem, oeomega

import numpy as np
import matplotlib.pyplot as plt
import matplotlib

In [87]:
def get_xtb(mol):
    qc_mol = mol.to_qcschema()

    xtb_model = Model(method="gfn2-xtb", basis=None)
    qc_task = AtomicInput(molecule=qc_mol, driver="energy", model=xtb_model)

    result = qcengine.compute(input_data=qc_task, program="xtb")
    
    # xtb returns energy in hartree
    return result.return_result * qcel.constants.conversion_factor("hartree", "kcal/mol")

def get_am1(mol):
    oe_mol = mol.to_openeye()
    
    calc = oequacpac.OEAM1()
    result = oequacpac.OEAM1Results()
    calc.CalcAM1(result, oe_mol)
    # am1 returns energy in kcal/mol
    return result.GetEnergy()

def getEnergies(sm_mol : str) -> ("xtb energy", "am1 energy"):
    molecule = 0
    
    try:
        molecule = Molecule.from_smiles(sm_mol, allow_undefined_stereo=True)
        molecule.generate_conformers()
    except Exception:
        return (None, None)
    
    return (get_am11(molecule), get_am1(molecule))
    

In [88]:
def get_conformers(openff_mol, max_conformers):
    mol = openff_mol.to_openeye()
    '''
    # set up omega
    omega = oeomega.OEOmega()
    omega.SetMaxConfs(max_conformers)
    omega.Build(mol)
    '''
    x = []
    for i in mol.GetConfs():
        # convert each conformer back to openff molecule
        ii = oechem.OEMol(i)
        aa = Molecule.from_openeye(ii, allow_undefined_stereo=True)
        
        x.append(aa)
        if len(x) == max_conformers:
            break
    return x

In [89]:
def calc_rmsd(vals):
    return np.sqrt(np.nanmean(vals**2,axis=0))

In [90]:
filename = "cmiles.smi"
mols = []
xtb_en = []
am1_en = []

ffmols = []

number = float("inf") # for every line
#number = 30
cnt = 0
pkldata = {}

with open(filename) as inputs:
    for line in inputs:
        if cnt >= number:
            break
        
        sm_mol = line.split()[0]
        
        if sm_mol == "isosmiles":
            continue
            
        try:
            molecule = Molecule.from_smiles(sm_mol, allow_undefined_stereo=True)
            molecule.generate_conformers()
            ffmols.append(get_conformers(molecule, 10))
            mols.append(sm_mol)
            pkldata[sm_mol] = {}
        except:
            print("skipped")
            continue
        cnt += 1
mols = np.array(mols)

In [94]:
b = np.full((10, len(ffmols)), np.nan)
one_conf = []
for i, confs in enumerate(ffmols):
    # log if only one conformer
    if len(confs) == 1:
        one_conf.append(mols[i])
 
        
    xtb_en = np.zeros((len(confs)))
    am1_en = np.zeros((len(confs)))
    for j, conf in enumerate(confs):
        xtb_en[j] = get_xtb(conf)
        am1_en[j] = get_am1(conf)
    
    # use lowest xtb energy as reference
    ref = list(xtb_en).index(min(xtb_en))
    xtb_c = xtb_en - xtb_en[ref]
    am1_c = am1_en - am1_en[ref]
    diff = xtb_c - am1_c
    
    pkldata[mols[i]] = {
                        "xtb_centered" : xtb_c,
                        "am1_centered" : am1_c,
                        "ref_idx" : ref,
                        "num_conformers" : len(confs),
                        "conformer_xtb_energies" : xtb_en,
                        "conformer_am1_energies" : am1_en,
                        "conformer_energy_difference" : diff
                       }

In [77]:
rmsd = calc_rmsd(b)
rmsd_signed = rmsd.copy()
for i in range(len(rmsd)):
    rmsd_signed[i] *= np.sign(pkldata[mols[i]]["xtb_mean"])*np.sign(pkldata[mols[i]]["am1_mean"])

    pkldata[mols[i]]["rmsd_energy_ratio"] = rmsd[i]
    pkldata[mols[i]]["rmsd_energy_ratio_signed"] = rmsd_signed[i]

In [95]:
# pickle the data
import pickle
'''
pickle format

{
    smiles string of molecule :
    {
        "spread" : float,                       # the difference between the largest and smallest energy ratios 
        "num_conformers" : int,                 # the number of conformations sampled, up to 10
        "conformer_xtb_energies" : [float],     # array of xtb-calculated energies for each conformer
        "conformer_am1_energies" : [float],     # array of am1-calculated energies for each conformer
        "conformer_energy_ratios" : [float],    # energy ratios
        "rmsd_energy_ratio" : float             # the average energy ratio
    }
}
'''
with open('xtb_am1_benchmark.pickle', 'wb') as pkfile:
    pickle.dump(pkldata, pkfile)

In [96]:
with open("one_conf.txt", "w") as oc_file:
    for x in one_conf:
        oc_file.write(x + "\n")